# Legal Analyzer Playground

This notebook lets you load two PDF documents from the `input/` folder, run the legal analyzer agentic graph, and explore the outputs.

---

In [ ]:
# 1. Import required modules, make sure you have installed the dependencies
import asyncio
from pathlib import Path
from ldaa.agents.graph import compiled_graph
from ldaa.schemas import LegalAnalysisState
from ldaa.utils.json import to_serializable
import json
from ldaa.utils.session import generate_session_id
import pdfplumber
import IPython.display as display



## 2. Set up input PDF paths
You can change these to point to any PDFs in the `input/` folder.

In [ ]:
doc1_path = '../input/ley1.pdf'  # Adjust path if needed
doc2_path = '../input/ley2.pdf'  # Adjust path if needed
output_path = '../output/playground_result.json'

print(f'Using PDF 1: {doc1_path}')
print(f'Using PDF 2: {doc2_path}')

## 3. Run the agentic graph
This cell will run the legal analyzer agentic graph on the two PDFs. This may take a few minutes depending on model and document size.

In [ ]:
   async def run_graph(doc1_path, doc2_path, output_path):
       state = LegalAnalysisState(doc1_path=doc1_path, doc2_path=doc2_path)
       thread_id = generate_session_id()
       print(f'Running agentic graph on: {doc1_path} and {doc2_path} (thread_id={thread_id})')
       result = await compiled_graph.ainvoke(state, configurable={"thread_id": thread_id})
       # Save result to output file
       Path(output_path).parent.mkdir(parents=True, exist_ok=True)
       with open(output_path, 'w') as f:
           json.dump(to_serializable(result), f, indent=2)
       print(f'Exported final result to {output_path}')
       return result

## 4. Explore the outputs
You can now explore the structured output, including per-paragraph analysis, comparative summary, and meta-logs.

In [ ]:
# Display the top-level keys
print('Top-level keys in result:', list(result.keys()))

# Show comparative summary if available
if result.comparison_result and result.comparison_result.comparative_summary:
    print('Comparative Summary:')
    display.display(result.comparison_result.comparative_summary)

# Show per-paragraph analysis for doc1
if result.doc1_analysis:
    print(f'
First 2 analyses for doc1:')
    for a in result.doc1_analysis[:2]:
        display.display(a)

# Show per-paragraph analysis for doc2
if result.doc2_analysis:
    print(f'
First 2 analyses for doc2:')
    for a in result.doc2_analysis[:2]:
        display.display(a)

# Show meta-logs
if result.meta_logs:
    print('Meta-logs:')
    display.display(result.meta_logs)
